# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [31]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

615

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
#Create base url
base_url = "http://api.openweathermap.org/data/2.5/weather?"



#Counters for weather loop
record = 0
sets = 1

#Create DataFrame
weather_df = pd.DataFrame({"City": cities})

weather_df["Latitude"] = ""
weather_df["Longitdue"] = ""
weather_df["Max Temp"] = ""
weather_df["Humidity"] = ""
weather_df["Cloudiness"] = ""
weather_df["Wind Speed"] = ""
weather_df["Country"] = ""
weather_df["Data"] = ""

#Iterate through Cities and retrieve data
print("Begining Data Retrieval")
print("--------------------------")

for index, row in weather_df.iterrows():
    
    city = row["City"]
    
    #Create a query url
    query_url = base_url + "appid=" + weather_api_key + "&q=" + city
    weather_response = requests.get(query_url).json()

    try:
        print(f"Processing Record {record} of Set {sets} | {city}.")

        weather_df.loc[index, "Country"] = weather_response["sys"]["country"]
        weather_df.loc[index, "Date"] = datetime.datetime.fronttimestamp(weather_response["dt"])
        weather_df.loc[index, "Latitude"] = weather_response["coord"]["lat"]
        weather_df.loc[index, "Longitude"] = weather_response["coord"]["lon"]
        weather_df.loc[index, "Cloudiness"] = weather_response["clouds"]["all"]
        weather_df.loc[index, "Humidity"] = weather_response["main"]["humidity"]
        weather_df.loc[index, "Max Temp"] = 1.8 * (weather_response["main"]["temp_max"] - 273) + 32
        weather_df.loc[index, "Wind Speed"] = weather_response["wind"]["speed"]
        
    except:
        print(f"City not found...skipping {city}.")
        
    record = record + 1
    if record == 50:
        record = 0
        sets = sets + 1

print("--------------------------")
print("Data Retrieval Complete")
print("--------------------------")

Begining Data Retrieval
--------------------------
Processing Record 0 of Set 1 | la ronge.
City not found...skipping la ronge.
Processing Record 1 of Set 1 | muros.
City not found...skipping muros.
Processing Record 2 of Set 1 | butaritari.
City not found...skipping butaritari.
Processing Record 3 of Set 1 | busselton.
City not found...skipping busselton.
Processing Record 4 of Set 1 | rikitea.
City not found...skipping rikitea.
Processing Record 5 of Set 1 | torbay.
City not found...skipping torbay.
Processing Record 6 of Set 1 | illoqqortoormiut.
City not found...skipping illoqqortoormiut.
Processing Record 7 of Set 1 | bluff.
City not found...skipping bluff.
Processing Record 8 of Set 1 | kot putli.
City not found...skipping kot putli.
Processing Record 9 of Set 1 | east london.
City not found...skipping east london.
Processing Record 10 of Set 1 | arraial do cabo.
City not found...skipping arraial do cabo.
Processing Record 11 of Set 1 | jaciara.
City not found...skipping jaciara.

Processing Record 3 of Set 3 | santiago del estero.
City not found...skipping santiago del estero.
Processing Record 4 of Set 3 | hihifo.
City not found...skipping hihifo.
Processing Record 5 of Set 3 | ribeira grande.
City not found...skipping ribeira grande.
Processing Record 6 of Set 3 | san quintin.
City not found...skipping san quintin.
Processing Record 7 of Set 3 | constitucion.
City not found...skipping constitucion.
Processing Record 8 of Set 3 | souillac.
City not found...skipping souillac.
Processing Record 9 of Set 3 | ilulissat.
City not found...skipping ilulissat.
Processing Record 10 of Set 3 | barrow.
City not found...skipping barrow.
Processing Record 11 of Set 3 | ostersund.
City not found...skipping ostersund.
Processing Record 12 of Set 3 | taganak.
City not found...skipping taganak.
Processing Record 13 of Set 3 | nampa.
City not found...skipping nampa.
Processing Record 14 of Set 3 | thompson.
City not found...skipping thompson.
Processing Record 15 of Set 3 | che

Processing Record 8 of Set 5 | mount isa.
City not found...skipping mount isa.
Processing Record 9 of Set 5 | santa maria del oro.
City not found...skipping santa maria del oro.
Processing Record 10 of Set 5 | nuzvid.
City not found...skipping nuzvid.
Processing Record 11 of Set 5 | barabai.
City not found...skipping barabai.
Processing Record 12 of Set 5 | tiksi.
City not found...skipping tiksi.
Processing Record 13 of Set 5 | pitea.
City not found...skipping pitea.
Processing Record 14 of Set 5 | ihosy.
City not found...skipping ihosy.
Processing Record 15 of Set 5 | kupang.
City not found...skipping kupang.
Processing Record 16 of Set 5 | yulara.
City not found...skipping yulara.
Processing Record 17 of Set 5 | gambela.
City not found...skipping gambela.
Processing Record 18 of Set 5 | izhma.
City not found...skipping izhma.
Processing Record 19 of Set 5 | port hedland.
City not found...skipping port hedland.
Processing Record 20 of Set 5 | guerrero negro.
City not found...skipping 

Processing Record 14 of Set 7 | tadine.
City not found...skipping tadine.
Processing Record 15 of Set 7 | leningradskiy.
City not found...skipping leningradskiy.
Processing Record 16 of Set 7 | atar.
City not found...skipping atar.
Processing Record 17 of Set 7 | turukhansk.
City not found...skipping turukhansk.
Processing Record 18 of Set 7 | alofi.
City not found...skipping alofi.
Processing Record 19 of Set 7 | salalah.
City not found...skipping salalah.
Processing Record 20 of Set 7 | arrecife.
City not found...skipping arrecife.
Processing Record 21 of Set 7 | robore.
City not found...skipping robore.
Processing Record 22 of Set 7 | chimbote.
City not found...skipping chimbote.
Processing Record 23 of Set 7 | ballarat.
City not found...skipping ballarat.
Processing Record 24 of Set 7 | curtea.
City not found...skipping curtea.
Processing Record 25 of Set 7 | saleaula.
City not found...skipping saleaula.
Processing Record 26 of Set 7 | rocha.
City not found...skipping rocha.
Proces

Processing Record 20 of Set 9 | aksarka.
City not found...skipping aksarka.
Processing Record 21 of Set 9 | beneditinos.
City not found...skipping beneditinos.
Processing Record 22 of Set 9 | banda aceh.
City not found...skipping banda aceh.
Processing Record 23 of Set 9 | coolum beach.
City not found...skipping coolum beach.
Processing Record 24 of Set 9 | giyani.
City not found...skipping giyani.
Processing Record 25 of Set 9 | nanortalik.
City not found...skipping nanortalik.
Processing Record 26 of Set 9 | wenling.
City not found...skipping wenling.


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression